# Final Notebook

This notebook is your search engine. 

For testing your work, we will run each cell. Thus, your code we'll have to fit the structure expected.



## Initialisation

- Install libraries (if you use Colab and needed),
- Import the modules,
- Declare global variable


In [8]:
FILEPATH = "datascience.stackexchange.com"

In [9]:
import pandas as pd
import numpy as np
import pickle as pkl
from math import log10,sqrt
import re
import tkinter as tk
import nltk


## Extraction of the data

In [10]:
def extract_data(filepath):
    return pd.read_xml(filepath, parser="etree", encoding="utf8")

In [11]:
# posts = extract_data(filepath=FILEPATH+"/Posts.xml")
users = extract_data(filepath=FILEPATH+"/Users.xml")
comments = extract_data(filepath=FILEPATH+"/Comments.xml")
votes = extract_data(filepath=FILEPATH+"/Votes.xml")
badges = extract_data(filepath=FILEPATH+"/Badges.xml")

ParseError: not well-formed (invalid token): line 1, column 29 (<string>)

## Indexation data

def index_data():
    # TODO
    
    return

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
# import re
def tokenize_data(string):
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    string = re.sub(r'[^\w\s]', '', string)
    return tokenizer.tokenize(str(string))

In [ ]:
# # tokenization with transformers bert uncased tokenizer

# # make posts smaller
# posts = posts.iloc[:1000, :]
# posts['Title'] = posts['Title'].apply(tokenize_data)
# posts['Body'] = posts['Body'].apply(tokenize_data)
# print(len(posts))
# load posts
file = "posts.pkl"
with open(file, 'rb') as f:
    posts = pkl.load(f)

In [ ]:
print(posts['Title'])

0        [information, retrieval, vs, recommendation, s...
1                                [the, idea, behind, drop]
2        [maximize, accuracy, with, differential, evolu...
3                                                       []
4                    [help, using, b, to, rank, sentences]
                               ...                        
75622    [best, way, to, detect, newly, incoming, an, i...
75623    [how, to, represent, and, graphical, display, ...
75624                               [rotate, vector, anti]
75625    [random, forest, partial, dependence, plot, ef...
75626    [can, get, in, a, p, data, return, a, random, ...
Name: Title, Length: 75627, dtype: object


In [ ]:
# If needed

def get_inverted_index_title(posts): # returns inverted index
    inverted_index = {}
    for i in range(len(posts)):
        for word in posts['Title'][i]:
            if word not in inverted_index:
                inverted_index[word] = {}
            if i not in inverted_index[word]:
                inverted_index[word][i] = 0
            inverted_index[word][i] += 1
    return inverted_index

def get_inverted_index_body(posts): # returns inverted index
    inverted_index = {}
    for i in range(len(posts)):
        for word in posts['Body'][i]:
            if word not in inverted_index:
                inverted_index[word] = {}
            if i not in inverted_index[word]:
                inverted_index[word][i] = 0
            inverted_index[word][i] += 1
    return inverted_index
  
inverted_index_body = get_inverted_index_body(posts)
inverted_index_title = get_inverted_index_title(posts)

In [ ]:
# #Save and Load your Index(es) in Pickle format
# def save_index(savepath):
#     with open(savepath, 'w') as file:
#          pkl.dump(data, file)
#          file.close()



# def load_index(savepath):
#     with open(savepath, 'r') as file:
#         data = pkl.load(file)
#         file.close()
#     return data

In [ ]:
def get_stop_words(inverted_index,k):
    stop_words = []
    for word in inverted_index:
        if len(inverted_index[word]) > len(posts)/k:
            stop_words.append(word)
    return stop_words

stop_words_body = get_stop_words(inverted_index_body,4)
stop_words_title = get_stop_words(inverted_index_title,20)
print(stop_words_body)
print(stop_words_title)
# remove stop words from inverted index
def remove_stop_words(inverted_index, stop_words):
    inverted_index_removed = {}
    for word in inverted_index:
        if word not in stop_words:
            inverted_index_removed[word] = inverted_index[word]
    return inverted_index_removed

inverted_index_body_removed = remove_stop_words(inverted_index_body, stop_words_body)
inverted_index_title_removed = remove_stop_words(inverted_index_title, stop_words_title)

# # remove stop words from posts
# def remove_stop_words_posts(posts, stop_words_body, stop_words_title):
#     posts_removed = posts.copy()
#     for i in range(len(posts_removed)):
#         posts_removed['Title'][i] = [word for word in posts_removed['Title'][i] if word not in stop_words_title]
#         posts_removed['Body'][i] = [word for word in posts_removed['Body'][i] if word not in stop_words_body]
#     return posts_removed

# posts = remove_stop_words_posts(posts, stop_words_body,stop_words_title)

['in', 'if', 'this', 'is', 'so', 'pi', 'have', 'a', 'hr', 're', 'nor', 'and', 'that', 'can', 'be', 'as', 'of', 'are', 'the', 'by', 'but', 'to', 'with', 'i', 'an', 'on', 'which', 'data', 'or', 'not', 'there', 'model', 'pre', 'code', 'for', 'p', 'it', 'you', 'would', 'like', 'pt', 'using', 'do', 'use', 'one', 'from', 'your']
['the', 'with', 'to', 'and', 'for', 'of', 'in', 'how', 'a', 'data', 'is']


In [ ]:
# Latent Semantic indexing


tfidfMatrix_body = np.zeros((len(inverted_index_body_removed), len(posts)))
for i, word in enumerate(inverted_index_body_removed):
    for j, doc in enumerate(inverted_index_body_removed[word]):
        tfidfMatrix_body[i,j] = (1 + log10(inverted_index_body_removed[word][doc])) * log10(len(posts)/len(inverted_index_body_removed[word]))

print(tfidfMatrix_body)

# build an SVD model, n_components = 100 is chosen in random
# svd_model = TruncatedSVD(n_components=100, 
#                          algorithm='randomized',
#                          n_iter=10, random_state=42)




KeyboardInterrupt: 

## Search Method

In [ ]:
def proba_search(query, df=posts,inverted_index, top=5):
  # each document get a score
  # OKAPI model (BM25)
  # print('query : ', query)
  # print('top : ', top)
  query_words = tokenize_data(query)
  k1 = 1.2
  b = 0.75
  k3 = 1000
  # average length of a document
  m = df['Body'].apply(lambda x: len(x)).mean()
  N = len(df)
  RSV_score = {}
  # for each post in df :
  for _, row in df.iterrows():
    # sum over all words in the query and in the post
    # length of the post
    Ld = len(df)
    # term frequency in the query
    def tf(word):
      return sum([1 for w in query_words if w == word])
    def d_f(word):
      if word not in inverted_index:
        #print(word)
        return 0
      else:
        return len(inverted_index[word])
    RSV_score[row['Id']] = sum([(k1+1)*tf(word)/(k1*((1-b)+b*Ld/m)+tf(word))*(k3+1)*tf(word)/(k3+tf(word))*np.log((N-d_f(word)+0.5)/(d_f(word)+0.5)) for word in query_words if word in row['words']])
  # return the top Ids of the posts from RSV
  sorted_keys = sorted(RSV_score, key=RSV_score.get, reverse=True) # the Id column in the best order
  # the values of sorted_keys are values of df["Id"]
  # return the top documents in the same order as sorted_keys
  new_df = df.copy()
  new_df['RSV_score'] = new_df['Id'].apply(lambda x: RSV_score[x])
  new_df = new_df.sort_values(by='RSV_score', ascending=False)
  return new_df[new_df["Id"].isin(sorted_keys[0:top])]
  #new_df[new_df["Id"].isin(sorted_keys[0:top])]

## Ranking

In [ ]:
#Il faut mettre le nombre de vues, le titre et les commentaires
def get_tags_from_postid(posts, id):
    taglist = posts[posts["Id"] == id]["Tags"]
    return taglist

def get_text(doc_id,posts):
    text = posts[posts["Id"] == doc_id]["Body"]
    return text

def get_reputation(doc_id,posts):
    # Réputation de l'auteur du post
    reputation = users[users["Id"] == int(posts[posts["Id"] == doc_id]["OwnerUserId"])]["Reputation"].values[0]
    return reputation
    
def get_title(doc_id,posts):
    title = posts[posts["Id"] == id]["Title"]
    return title

def get_RSV_score(query, df,inverted_index, top,doc_id):
    dataf=proba_search(query,df,inverted_index,top)
    RSV_score=dataf[dataf["Id"]==doc_id]['RSV_score']
    return RSV_score


def doc_tags_association(posts):
    association = []
    for i in range(0, len(posts)):
        association += [(i, get_tags_from_postid(posts, i))]
    return association

def get_votes(posts, id):
    association = doc_tags_association(posts)
    return posts[posts["Id"] == association[posts]]

def tokenize_avec_repetition(text):
    t = text.lower()
    t = re.sub(r"[^\w\s]", " ", t)
    return t.split()

def vectorize_element(element, vocabulary):
    n = len(vocabulary)
    vecteur = np.zeros((1, n))
    element_tokens = tokenize_avec_repetition(element)
    key_list = list(vocabulary.keys())
    for elem in element_tokens:
        i = key_list.index(elem)
        vecteur[0][i] += 1
    return vecteur

def match(query, doc_id, vocabulary, pcosbody, pcostitle, ptags, pvotes, pvues, pcommentaires, preputation):
    rcosdoc = mesure_cos_element(query,element,vocabulary)
    vectq=np.array(vectorize_element(query,vocabulary))

    dtags = get_tags_from_postid(posts, doc_id)
    rtags = mesure_cos_element(query, dtags, vocabulary)

    for i in range(0, len(vectq)):
        for j in range(0, len(dtags)):
            if vectq[i] == dtags[j]:
                rtags += 1

    reputation = log10( 1 + get_reputation(doc_id, posts))

    rvotes = log10( 1 + get_votes(posts, doc_id))
    # on prend le log car un score avec 10000 likes n'est pas forct 100 fois plus pertinent qu'un post avec  d0 likes

    return pcos * rcos + ptags * rtags + pvotes * rvotes + preputation * reputation
''''
def search(query, top =10)
    for i in range (0,len(posts)): 
        classement += [(i,atch( query_tk, i , pcos= 1, ptags=1, pvotes=1))]

    
    classement_trie = sorted(classement, key=lambda x: x[1])

    classement = []
    # take the top doc_id with the highest match score with the query
    for i in range(0, top):
        classement += [classement[i][0]]


    return classement
'''

def search(query, top =10):
    return query

In [ ]:
print(get_text(123,posts))

9298    [pt, most, basic, relationship, to, describe, ...
Name: Body, dtype: object


## Visualising Results

In [7]:
import tkinter as tk

def search(query, top =10):
    return query

# Tkinter interface for the search engine
# create a window
window = Tk()
window.title("Search Engine")
window.geometry('1000x600')
# create a label
lbl = Label(window, text="Search Engine", font=("Arial Bold", 50))
lbl.grid(column=0, row=0)
# create a label
lbl = Label(window, text="Enter your query", font=("Arial Bold", 20))
lbl.grid(column=0, row=1)
# create a text entry box
txt = Entry(window, width=100)
txt.grid(column=0, row=2)
# call the fonction search when the button is clicked
def clicked():
    query = txt.get()
    search(query, top=10)

    

NameError: name 'Tk' is not defined

## Querying

## Scoring

In [11]:
# Pas sûr de garder cette partie

In [12]:
# TODO